# IMPORTING LIBRARY

In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from sklearn.decomposition import TruncatedSVD

# LOADING DATA


In [4]:
movies_data=pd.read_csv("D:\minor_2\ml-latest\ml-latest\movies.csv")

In [5]:
ratings_data=pd.read_csv(r"D:\minor_2\ml-latest\ml-latest\ratings.csv")

In [6]:
tags_data=pd.read_csv(r"D:\minor_2\ml-latest\ml-latest\tags.csv")

# VIEWING DATA

In [7]:
movies_data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [9]:
ratings_data.head(5)

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [10]:
tags_data.head(5)

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


# REMOVING OUTLIERS

In [11]:
ratings_data.pop("timestamp")

0           1256677221
1           1256677456
2           1256677471
3           1256677460
4           1256677264
               ...    
27753439    1379882795
27753440    1379882751
27753441    1379882889
27753442    1354159524
27753443    1354159718
Name: timestamp, Length: 27753444, dtype: int64

In [12]:
tags_data.pop("timestamp")

0          1443148538
1          1443148532
2          1442169410
3          1442169421
4          1442615195
              ...    
1108992    1264379059
1108993    1264379058
1108994    1264379058
1108995    1264379058
1108996    1168346830
Name: timestamp, Length: 1108997, dtype: int64

# SOME STATISTICS

In [13]:
a=movies_data["movieId"].unique()
print(len(a))

58098


In [14]:
a=ratings_data["movieId"].unique()
print(len(a))

53889


In [15]:
a=tags_data["movieId"].unique()
print(len(a))

45981


In [16]:
a=ratings_data["userId"].unique()
print(len(a))

283228


In [17]:
a=tags_data["userId"].unique()
print(len(a))

19325


# PREPARING DATA 

In [18]:
movies_data["genres"]=movies_data["genres"].str.replace('|'," ")

<ipython-input-18-e4922e2b3002>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  movies_data["genres"]=movies_data["genres"].str.replace('|'," ")


In [19]:
movies_data

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure Drama Horror Sci-Fi


In [20]:
ratings_data_f=ratings_data.groupby("userId").filter(lambda x: len(x)>=20)

In [21]:
ratings_data_f

,userId,movieId,rating
42,4,1,4.0
43,4,2,4.0
44,4,5,2.0
45,4,6,4.5
46,4,10,4.0
...,...,...,...
27753439,283228,8542,4.5
27753440,283228,8712,4.5
27753441,283228,34405,4.5
27753442,283228,44761,4.5


In [22]:
movie_list_rating=ratings_data_f.movieId.unique().tolist()

In [23]:
movie_data=movies_data[movies_data.movieId.isin(movie_list_rating)]

In [24]:
content_data=pd.merge(movies_data,tags_data,on="movieId",how="left")

In [25]:
content_data.head()

,movieId,title,genres,userId,tag
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,animated
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,buddy movie
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,Cartoon
3,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,cgi
4,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,comedy


In [26]:
content_data.fillna("",inplace=True)
content_data=pd.DataFrame(content_data.groupby("movieId")["tag"].apply(lambda x: "%s" % " ".join(x)))

In [27]:
final=pd.merge(movies_data,content_data,on="movieId",how="left")

In [28]:
final

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,animated buddy movie Cartoon cgi comedy comput...
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy adapted from:book animals bad cgi base...
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old Ann Margaret Burgess Meredith Daryl ...
3,4,Waiting to Exhale (1995),Comedy Drama Romance,characters girl movie characters chick flick b...
4,5,Father of the Bride Part II (1995),Comedy,steve martin steve martin pregnancy remake agi...
...,...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed),
58094,193878,Les tribulations d'une caissière (2011),Comedy,
58095,193880,Her Name Was Mumu (2016),Drama,
58096,193882,Flora (2017),Adventure Drama Horror Sci-Fi,


In [29]:
final["metadata"]=final[["tag","genres"]].apply(lambda x: " ".join(x),axis=1)

In [30]:
final.head()

,movieId,title,genres,tag,metadata
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,animated buddy movie Cartoon cgi comedy comput...,animated buddy movie Cartoon cgi comedy comput...
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy adapted from:book animals bad cgi base...,fantasy adapted from:book animals bad cgi base...
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old Ann Margaret Burgess Meredith Daryl ...,moldy old Ann Margaret Burgess Meredith Daryl ...
3,4,Waiting to Exhale (1995),Comedy Drama Romance,characters girl movie characters chick flick b...,characters girl movie characters chick flick b...
4,5,Father of the Bride Part II (1995),Comedy,steve martin steve martin pregnancy remake agi...,steve martin steve martin pregnancy remake agi...


In [31]:
print(final["metadata"].iloc(0)[1])

fantasy adapted from:book animals bad cgi based on a book board game childhood recaptured Children Chris Van Allsburg fantasy Filmed in BC jungle kid flick Kirsten Dunst monkey Robin Williams Saturn Award (Best Special Effects) Saturn Award (Best Supporting Actress) scary time time travel fantasy Robin Williams adapted from:book childish Children kid flick time travel Robin Williams time travel Robin Williams Joe Johnston Robin Williams For children kid flick itaege fantasy Robin Williams scary time travel game animals comedy fiction thrill Dynamic CGI Action bad cgi horrifying (but not Horror genre) Kirsten Dunst magic board game monkey not for kids based on children's book board game disappearance giant insect new home recluse animals fantasy Lebbat bad cgi magic board game Children Fantasy Robin Williams Kirsten Dunst Robin Williams Robin Williams animals based on a book fantasy magic board game monkey not for kids Robin Williams scary time travel animals bad cgi board game not for 

In [32]:
tfidf=TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf.fit_transform(final["metadata"])
tfidf_df=pd.DataFrame(tfidf_matrix.toarray(),index=final.index.tolist())
print(tfidf_df.shape)

(58098, 36418)


In [33]:
svd=TruncatedSVD(n_components=200)
latent_matrix=svd.fit_transform(tfidf_df)

MemoryError: Unable to allocate 15.8 GiB for an array with shape (58098, 36418) and data type float64

In [ ]:
explained=svd.explained_variance_ratio_.cumsum()

# processing
